In [1]:
from datetime import timedelta
import warnings


import polars as pl
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import RobustScaler
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
import joblib
import optuna
from datetime import datetime

device = "cuda" if torch.cuda.is_available() else "cpu"

warnings.filterwarnings("ignore")

c:\Users\Mi\Документы\GitHub\AITH-Blockchain-AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device

'cpu'

## Test dataset

Let's calculate metrics on the first 5 minutes of the test part

In [3]:
test_df_for_net_1h = pl.read_parquet("../data/processed/test.parquet")
print(test_df_for_net_1h.shape)

(75570, 14)


In [4]:
test_df_for_net_5m = test_df_for_net_1h[:7348] # Количество сэмплов, соответствующее промежутку времени длиной 5 минут
print(test_df_for_net_5m.shape)

(7348, 14)


## Models Architectures

In [5]:
class SequenceDataset(Dataset):
    def __init__(self, df: pl.DataFrame, seq_length: int, target_col: str):
        self.seq_length = seq_length
        feature_columns = [col for col in df.columns if col != target_col]
        self.features = df.select(feature_columns).to_numpy()
        self.targets = df[target_col].to_numpy()

    def __len__(self):
        return self.features.shape[0] - self.seq_length

    def __getitem__(self, idx):
        seq_x = self.features[idx : idx + self.seq_length]
        target = self.targets[idx + self.seq_length]
        return torch.tensor(seq_x, dtype=torch.float32), torch.tensor([target], dtype=torch.float32)

In [6]:
class BasicRNN(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        output_size,
        dropout_rate=0.5,
    ):
        super(BasicRNN, self).__init__()
        self.rnn = nn.RNN(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.rnn.num_layers, x.size(0), self.rnn.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out


class BasicLSTM(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        output_size,
        dropout_rate=0.5,
    ):
        super(BasicLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out


class BasicBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate=0.5):
        super(BasicBiLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0,
            bidirectional=True,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers * 2, x.size(0), self.lstm.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out


class CNNLSTM(nn.Module):
    def __init__(
        self,
        input_size,
        conv_filters=9,
        kernel_size=7,
        lstm_hidden_size=100,
        lstm_num_layers=2,
        output_size=1,
        dropout_rate=0.5,
        padding=True,
    ):
        super(CNNLSTM, self).__init__()
        self.input_size = input_size

        pad = kernel_size // 2 if padding else 0
        self.conv1d = nn.Conv1d(
            in_channels=input_size,
            out_channels=input_size * conv_filters,
            kernel_size=kernel_size,
            groups=input_size,
            padding=pad,
        )

        self.lstm = nn.LSTM(
            input_size=input_size * conv_filters,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_num_layers,
            batch_first=True,
            dropout=dropout_rate,
        )
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(lstm_hidden_size, output_size)

    def forward(self, x):
        x = x.transpose(1, 2)
        conv_out = torch.tanh(self.conv1d(x))
        conv_out = conv_out.transpose(1, 2)
        batch_size = conv_out.size(0)
        h0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size, device=x.device)
        c0 = torch.zeros(self.lstm.num_layers, batch_size, self.lstm.hidden_size, device=x.device)
        lstm_out, _ = self.lstm(conv_out, (h0, c0))
        out = lstm_out[:, -1, :]
        out = self.dropout(out)
        out = self.fc(out)
        return out


class AttentionLayer(nn.Module):
    def __init__(self, enc_size, dec_size, attn_hidden_size):
        super().__init__()
        self.linear_enc = nn.Linear(enc_size, attn_hidden_size)
        self.linear_dec = nn.Linear(dec_size, attn_hidden_size)
        self.linear_out = nn.Linear(attn_hidden_size, 1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, enc, dec, inp_mask):
        batch_size, seq_length, _ = enc.shape
        query_proj = self.linear_dec(dec).unsqueeze(1)  # (batch, 1, attn_hidden_size)
        enc_proj = self.linear_enc(enc)  # (batch, seq_length, attn_hidden_size)
        energies = torch.tanh(enc_proj + query_proj)  # (batch, seq_length, attn_hidden_size)
        scores = self.linear_out(energies).squeeze(-1)  # (batch, seq_length)

        scores = scores.masked_fill(~inp_mask, -1e9)

        attn_weights = self.softmax(scores)  # (batch, seq_length)
        context = torch.sum(attn_weights.unsqueeze(-1) * enc, dim=1)  # (batch, enc_size)
        return context, attn_weights


class AttentionLSTM(nn.Module):
    def __init__(
        self,
        input_size,
        hidden_size,
        num_layers,
        attn_hidden_size,
        output_size=1,
        dropout_rate=0.5,
    ):
        super(AttentionLSTM, self).__init__()
        self.encoder = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0,
        )
        self.attention = AttentionLayer(hidden_size, hidden_size, attn_hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, mask=None):
        encoder_outputs, (h_n, c_n) = self.encoder(x)
        query = h_n[-1]  # (batch, hidden_size)

        if mask is None:
            mask = torch.ones(encoder_outputs.size(0), encoder_outputs.size(1), dtype=torch.bool, device=x.device)

        context, attn_weights = self.attention(encoder_outputs, query, mask)
        out = self.dropout(context)
        out = self.fc(out)
        return out

## Methods for evaluation

In [7]:
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions = []
    targets = []
    with torch.no_grad():
        for seq, target in dataloader:
            seq = seq.to(device)
            target = target.to(device)
            output = model(seq)
            predictions.append(output.cpu().numpy())
            targets.append(target.cpu().numpy())

    predictions = np.concatenate(predictions, axis=0)
    targets = np.concatenate(targets, axis=0)
    return targets, predictions


def plot_predictions(y_true, y_pred, title="Predicted vs Actual Gas Price"):
    plt.figure(figsize=(10, 5))
    plt.plot(y_true, label="Actual")
    plt.plot(y_pred, label="Predicted")
    plt.title(title)
    plt.xlabel("Time step")
    plt.ylabel("Gas Price")
    plt.legend()
    plt.show()


def calculate_metrics(y_true, y_pred):
    epsilon = 1e-8
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100
    mae = np.mean(np.abs(y_true - y_pred))
    return {"MAPE": mape, "MAE": mae}


def inverse_scale(scaled_data, scaler):
    return scaler.inverse_transform(scaled_data)

In [8]:
scaler = joblib.load("../models/tx_scaler.pkl")
numeric_cols = ['cost', 'gas', 'gas_fee_cap', 'gas_price']

test_df_numeric = test_df_for_net_1h.select(numeric_cols)
test_df_numeric_pd = test_df_numeric.to_pandas().astype(float)

inverted_data = inverse_scale(test_df_numeric_pd, scaler)

y_true_unscaled = inverted_data[:, 3].copy()

## Common parameters

In [9]:
seq_length = 50
target_column = "gas_price"
batch_size = 64

test_dataset_5m = SequenceDataset(test_df_for_net_5m, seq_length, target_column)
test_dataset_1h = SequenceDataset(test_df_for_net_1h, seq_length, target_column)

In [10]:
test_loader_5m = DataLoader(test_dataset_5m, batch_size=batch_size, shuffle=False)
test_loader_1h = DataLoader(test_dataset_1h, batch_size=batch_size, shuffle=False)

In [11]:
input_size = test_df_for_net_5m.width - 1
output_size = 1

## Models instances 

In [12]:
def load_model(model_type, model_path, device, **model_kwargs):
    model = model_type(**model_kwargs)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

### RNN

In [13]:
hidden_size_rnn = input_size * 4
num_layers_rnn = 2

In [ ]:
rnn_kwargs = {
    "input_size": input_size,
    "hidden_size": hidden_size_rnn,
    "num_layers": num_layers_rnn,
    "output_size": output_size,
}

model_rnn = load_model(BasicRNN, "../models/rnn/best_model.pt", device, **rnn_kwargs)

In [ ]:
y_true_rnn_5m, y_pred_rnn_5m = evaluate_model(model_rnn, test_loader_5m, device)
y_true_rnn_1h, y_pred_rnn_1h = evaluate_model(model_rnn, test_loader_1h, device)

In [ ]:
y_pred_rnn_matrix_1h = inverted_data[50:, :].copy()
y_pred_rnn_matrix_1h[:, 3] = y_pred_rnn_1h.T[0].copy()

y_pred_rnn_unscaled_1h = inverse_scale(y_pred_rnn_matrix_1h, scaler)[:, 3].copy()

metrics_rnn_unscaled_1h = calculate_metrics(y_true_unscaled[50:], y_pred_rnn_unscaled_1h)
print(metrics_rnn_unscaled_1h)

{'MAPE': np.float32(93.52057), 'MAE': np.float32(2.1094285e-08)}


In [ ]:
y_pred_rnn_matrix_5m = inverted_data[50:7348, :].copy()
y_pred_rnn_matrix_5m[:, 3] = y_pred_rnn_5m.T[0].copy()

y_pred_rnn_unscaled_5m = inverse_scale(y_pred_rnn_matrix_5m, scaler)[:, 3].copy()

metrics_rnn_unscaled_5m = calculate_metrics(y_true_unscaled[50:7348], y_pred_rnn_unscaled_5m)
print(metrics_rnn_unscaled_5m)

{'MAPE': np.float32(95.23118), 'MAE': np.float32(2.5627902e-08)}


### LSTM

In [15]:
hidden_size_lstm = input_size * 4
num_layers_lstm = 2

In [16]:
lstm_kwargs = {
    "input_size": input_size,
    "hidden_size": hidden_size_lstm,
    "num_layers": num_layers_lstm,
    "output_size": output_size,
}

model_lstm = load_model(BasicLSTM, "../models/lstm/best_model.pt", device, **lstm_kwargs)

In [17]:
y_true_lstm_5m, y_pred_lstm_5m = evaluate_model(model_lstm, test_loader_5m, device)
y_true_lstm_1h, y_pred_lstm_1h = evaluate_model(model_lstm, test_loader_1h, device)

In [18]:
y_pred_lstm_matrix_1h = inverted_data[50:, :].copy()
y_pred_lstm_matrix_1h[:, 3] = y_pred_lstm_1h.T[0].copy()

y_pred_lstm_unscaled_1h = inverse_scale(y_pred_lstm_matrix_1h, scaler)[:, 3]

metrics_lstm_unscaled_1h = calculate_metrics(y_true_unscaled[50:], y_pred_lstm_unscaled_1h)
print(metrics_lstm_unscaled_1h)

{'MAPE': np.float64(91.15660468230567), 'MAE': np.float64(2.087373620507348e-08)}


In [19]:
y_pred_lstm_matrix_5m = inverted_data[50:7348, :].copy()
y_pred_lstm_matrix_5m[:, 3] = y_pred_lstm_5m.T[0].copy()

y_pred_lstm_unscaled_5m = inverse_scale(y_pred_lstm_matrix_5m, scaler)[:, 3]

metrics_lstm_unscaled_5m = calculate_metrics(y_true_unscaled[50:7348], y_pred_lstm_unscaled_5m)
print(metrics_lstm_unscaled_5m)

{'MAPE': np.float64(93.12762814623254), 'MAE': np.float64(2.1351191925552063e-08)}


### BiLSTM

In [20]:
hidden_size_bi_lstm = input_size * 4
num_layers_bi_lstm = 2

In [21]:
bi_lstm_kwargs = {
    "input_size": input_size,
    "hidden_size": hidden_size_bi_lstm,
    "num_layers": num_layers_bi_lstm,
    "output_size": output_size,
}

model_bi_lstm = load_model(BasicBiLSTM, "../models/bi_lstm/best_model.pt", device, **bi_lstm_kwargs)

In [22]:
y_true_bi_lstm_5m, y_pred_bi_lstm_5m = evaluate_model(model_bi_lstm, test_loader_5m, device)
y_true_bi_lstm_1h, y_pred_bi_lstm_1h = evaluate_model(model_bi_lstm, test_loader_1h, device)

In [23]:
y_pred_bi_lstm_matrix_1h = inverted_data[50:, :].copy()
y_pred_bi_lstm_matrix_1h[:, 3] = y_pred_bi_lstm_1h.T[0].copy()

y_pred_bi_lstm_unscaled_1h = inverse_scale(y_pred_bi_lstm_matrix_1h, scaler)[:, 3]

metrics_bi_lstm_unscaled_1h = calculate_metrics(y_true_unscaled[50:], y_pred_bi_lstm_unscaled_1h)
print(metrics_bi_lstm_unscaled_1h)

{'MAPE': np.float64(82.89724787302563), 'MAE': np.float64(1.9977121431888418e-08)}


In [24]:
y_pred_bi_lstm_matrix_5m = inverted_data[50:7348, :].copy()
y_pred_bi_lstm_matrix_5m[:, 3] = y_pred_bi_lstm_5m.T[0].copy()

y_pred_bi_lstm_unscaled_5m = inverse_scale(y_pred_bi_lstm_matrix_5m, scaler)[:, 3]

metrics_bi_lstm_unscaled_5m = calculate_metrics(y_true_unscaled[50:7348], y_pred_bi_lstm_unscaled_5m)
print(metrics_bi_lstm_unscaled_5m)

{'MAPE': np.float64(83.78664627321803), 'MAE': np.float64(2.0387802073476727e-08)}


### CNN LSTM

In [25]:
lstm_cnn_conv_filters = 9
lstm_cnn_kernel_size = 7
lstm_cnn_hidden_size = input_size * 4
lstm_cnn_num_layers = 2

In [26]:
cnn_lstm_kwargs = {
    "input_size": input_size,
    "conv_filters": lstm_cnn_conv_filters,
    "kernel_size": lstm_cnn_kernel_size,
    "lstm_hidden_size": lstm_cnn_hidden_size,
    "lstm_num_layers": lstm_cnn_num_layers,
    "output_size": output_size,
}

model_cnn_lstm = load_model(CNNLSTM, "../models/cnn_lstm/best_model.pt", device, **cnn_lstm_kwargs)

In [27]:
y_true_cnn_lstm_5m, y_pred_cnn_lstm_5m = evaluate_model(model_cnn_lstm, test_loader_5m, device)
y_true_cnn_lstm_1h, y_pred_cnn_lstm_1h = evaluate_model(model_cnn_lstm, test_loader_1h, device)

In [28]:
y_pred_cnn_lstm_matrix_1h = inverted_data[50:, :].copy()
y_pred_cnn_lstm_matrix_1h[:, 3] = y_pred_cnn_lstm_1h.T[0].copy()

y_pred_cnn_lstm_unscaled_1h = inverse_scale(y_pred_cnn_lstm_matrix_1h, scaler)[:, 3]

metrics_cnn_lstm_unscaled_1h = calculate_metrics(y_true_unscaled[50:], y_pred_cnn_lstm_unscaled_1h)
print(metrics_cnn_lstm_unscaled_1h)

{'MAPE': np.float64(84.60640292915505), 'MAE': np.float64(2.0167606517621747e-08)}


In [29]:
y_pred_cnn_lstm_matrix_5m = inverted_data[50:7348, :].copy()
y_pred_cnn_lstm_matrix_5m[:, 3] = y_pred_cnn_lstm_5m.T[0].copy()

y_pred_cnn_lstm_unscaled_5m = inverse_scale(y_pred_cnn_lstm_matrix_5m, scaler)[:, 3]

metrics_cnn_lstm_unscaled_5m = calculate_metrics(y_true_unscaled[50:7348], y_pred_cnn_lstm_unscaled_5m)
print(metrics_cnn_lstm_unscaled_5m)

{'MAPE': np.float64(85.80621719315685), 'MAE': np.float64(2.0720682075807393e-08)}


### Attention LSTM

In [30]:
hidden_size_attention_lstm = input_size * 4
num_layers_attention_lstm = 2

In [31]:
attention_lstm_kwargs = {
    "input_size": input_size,
    "hidden_size": hidden_size_attention_lstm,
    "num_layers": num_layers_attention_lstm,
    "attn_hidden_size": output_size,
    "output_size": output_size,
}

model_attention_lstm = load_model(AttentionLSTM, "../models/attention_lstm/best_model.pt", device, **attention_lstm_kwargs)

In [32]:
y_true_attention_lstm_5m, y_pred_attention_lstm_5m = evaluate_model(model_attention_lstm, test_loader_5m, device)
y_true_attention_lstm_1h, y_pred_attention_lstm_1h = evaluate_model(model_attention_lstm, test_loader_1h, device)

In [33]:
y_pred_attention_lstm_matrix_1h = inverted_data[50:, :].copy()
y_pred_attention_lstm_matrix_1h[:, 3] = y_pred_attention_lstm_1h.T[0].copy()

y_pred_attention_lstm_unscaled_1h = inverse_scale(y_pred_attention_lstm_matrix_1h, scaler)[:, 3]

metrics_attention_lstm_unscaled_1h = calculate_metrics(y_true_unscaled[50:], y_pred_attention_lstm_unscaled_1h)
print(metrics_attention_lstm_unscaled_1h)

{'MAPE': np.float64(83.26548750524651), 'MAE': np.float64(2.0058201909877575e-08)}


In [34]:
y_pred_attention_lstm_matrix_5m = inverted_data[50:7348, :].copy()
y_pred_attention_lstm_matrix_5m[:, 3] = y_pred_attention_lstm_5m.T[0].copy()

y_pred_attention_lstm_unscaled_5m = inverse_scale(y_pred_attention_lstm_matrix_5m, scaler)[:, 3]

metrics_attention_lstm_unscaled_5m = calculate_metrics(y_true_unscaled[50:7348], y_pred_attention_lstm_unscaled_5m)
print(metrics_attention_lstm_unscaled_5m)

{'MAPE': np.float64(83.6178218074625), 'MAE': np.float64(2.0446592508697535e-08)}


Let's prepare test data for `Prophet` and `ARIMA`

In [35]:
df_full_ticks = pl.read_parquet("../data/processed/tx_blocks_eth_clean.parquet")

In [36]:
max_time = df_full_ticks["tx_time"].max()
test_threshold = max_time - timedelta(hours=1)
val_threshold = test_threshold - timedelta(hours=1)

test_threshold, val_threshold

(datetime.datetime(2025, 4, 13, 12, 43, 23),
 datetime.datetime(2025, 4, 13, 11, 43, 23))

In [37]:
df_blocks = df_full_ticks.group_by("block_hash").agg(pl.col("tx_time").min().alias("block_time"))

In [38]:
df = df_full_ticks.join(df_blocks, on="block_hash", how="left")

In [39]:
test_df_1h = df.filter(pl.col("block_time") >= test_threshold)
test_df_1h.shape

(75570, 20)

In [40]:
min_test_time = test_df_1h["block_time"].min()
threshold = min_test_time + timedelta(minutes=5)

In [41]:
test_df_5m = test_df_1h.filter(pl.col("block_time") < threshold)
print(test_df_5m.shape)

(7348, 20)


In [42]:
def add_pseudo_time(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy(deep=True)

    df_copy["block_time"] = pd.to_datetime(df_copy["block_time"], unit="s")
    df_copy["block_timestamp"] = df_copy["block_time"].astype("int") // 10**6

    df_copy["n_in_block"] = df_copy.groupby("block_hash")["block_hash"].transform("count")
    df_copy["tx_index_in_block"] = df_copy.groupby("block_hash").cumcount()

    df_copy["tx_timestamp"] = df_copy["block_timestamp"] + df_copy["tx_index_in_block"] / df_copy["n_in_block"]

    df_copy["tx_time"] = pd.to_datetime(df_copy["tx_timestamp"], unit="s")
    
    df_copy = df_copy.drop(columns=[
        "block_timestamp", "tx_index_in_block", "n_in_block", "tx_timestamp"
    ])

    return df_copy

In [43]:
test_df_pd_1h = test_df_1h.select(["gas_price", "block_time", "block_hash"]).to_pandas()

test_df_pd_1h = add_pseudo_time(test_df_pd_1h)
test_df_pd_1h.head()

,gas_price,block_time,block_hash,tx_time
0,-0.159582,2025-04-13 13:22:59,0x8dde0f4ac2de2c39f15094639fb4f18bc61e57fefba7...,2025-04-13 13:22:59.000000000
1,0.419496,2025-04-13 13:22:59,0x8dde0f4ac2de2c39f15094639fb4f18bc61e57fefba7...,2025-04-13 13:22:59.002146006
2,256.684807,2025-04-13 13:22:59,0x8dde0f4ac2de2c39f15094639fb4f18bc61e57fefba7...,2025-04-13 13:22:59.004291773
3,256.684807,2025-04-13 13:22:59,0x8dde0f4ac2de2c39f15094639fb4f18bc61e57fefba7...,2025-04-13 13:22:59.006437778
4,2.197437,2025-04-13 13:22:59,0x8dde0f4ac2de2c39f15094639fb4f18bc61e57fefba7...,2025-04-13 13:22:59.008583784


In [44]:
test_df_pd_5m = test_df_5m.select(["gas_price", "block_time", "block_hash"]).to_pandas()

test_df_pd_5m = add_pseudo_time(test_df_pd_5m)
test_df_pd_5m.head()

,gas_price,block_time,block_hash,tx_time
0,-0.394365,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.000000000
1,-0.368419,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.003676414
2,-0.368403,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.007352829
3,-0.382128,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.011029482
4,-0.368403,2025-04-13 12:48:11,0xa2267aa8f7c058206cbda95258ac59dfe8639875e2b2...,2025-04-13 12:48:11.014705896


### Prophet

In [45]:
prophet_model = joblib.load("../models/prophet/prophet_model.pkl")

In [46]:
prophet_forecast_5m = prophet_model.predict(test_df_pd_5m[["tx_time"]].rename(columns={"tx_time": "ds"}))
prophet_forecast_1h = prophet_model.predict(test_df_pd_1h[["tx_time"]].rename(columns={"tx_time": "ds"}))

In [47]:
y_true_prophet_1h = test_df_pd_1h["gas_price"].to_numpy()
y_pred_prophet_1h = prophet_forecast_1h["yhat"].to_numpy()

In [48]:
y_true_prophet_5m = test_df_pd_5m["gas_price"].to_numpy()
y_pred_prophet_5m = prophet_forecast_5m["yhat"].to_numpy()

In [53]:
y_pred_prophet_matrix_1h = inverted_data[50:, :].copy()
y_pred_prophet_matrix_1h[:, 3] = y_pred_prophet_1h.T[0].copy()

y_pred_prophet_unscaled_1h = inverse_scale(y_pred_prophet_matrix_1h, scaler)[:, 3]

metrics_prophet_unscaled_1h = calculate_metrics(y_true_unscaled[50:], y_pred_prophet_unscaled_1h)
print(metrics_prophet_unscaled_1h)

{'MAPE': np.float64(149.58715131845756), 'MAE': np.float64(2.912236401877894e-08)}


In [54]:
y_pred_prophet_matrix_5m = inverted_data[50:7348, :].copy()
y_pred_prophet_matrix_5m[:, 3] = y_pred_prophet_5m.T[0].copy()

y_pred_prophet_unscaled_5m = inverse_scale(y_pred_prophet_matrix_5m, scaler)[:, 3]

metrics_prophet_unscaled_5m = calculate_metrics(y_true_unscaled[50:7348], y_pred_prophet_unscaled_5m)
print(metrics_prophet_unscaled_5m)

{'MAPE': np.float64(150.5233931406163), 'MAE': np.float64(2.938325793366939e-08)}


### ARIMA

In [51]:
arima_model = joblib.load("../models/arima/arima_model.pkl")

In [52]:
arima_forecast_5m = arima_model.forecast(steps=test_df_pd_5m.shape[0])
arima_forecast_1h = arima_model.forecast(steps=test_df_pd_1h.shape[0])

In [55]:
y_true_arima_1h = test_df_pd_1h["gas_price"].to_numpy()
y_pred_arima_1h = np.array(arima_forecast_1h)

In [56]:
y_true_arima_5m = test_df_pd_5m["gas_price"].to_numpy()
y_pred_arima_5m = np.array(arima_forecast_5m)

In [58]:
y_pred_arima_matrix_1h = inverted_data[50:, :].copy()
y_pred_arima_matrix_1h[:, 3] = y_pred_arima_1h.T[0].copy()

y_pred_arima_unscaled_1h = inverse_scale(y_pred_arima_matrix_1h, scaler)[:, 3]

metrics_arima_unscaled_1h = calculate_metrics(y_true_unscaled[50:], y_pred_arima_unscaled_1h)
print(metrics_arima_unscaled_1h)

{'MAPE': np.float64(295.2359637165212), 'MAE': np.float64(4.455106689524225e-08)}


In [59]:
y_pred_arima_matrix_5m = inverted_data[50:7348, :].copy()
y_pred_arima_matrix_5m[:, 3] = y_pred_arima_5m.T[0].copy()

y_pred_arima_unscaled_5m = inverse_scale(y_pred_arima_matrix_5m, scaler)[:, 3]

metrics_arima_unscaled_5m = calculate_metrics(y_true_unscaled[50:7348], y_pred_arima_unscaled_5m)
print(metrics_arima_unscaled_5m)

{'MAPE': np.float64(296.80281279518096), 'MAE': np.float64(4.4827176037002904e-08)}


## Final Comparison

Let's check metrics on the unscaled data the next 1 hour

In [72]:
results_unscaled_1h = {
    # "rnn": metrics_rnn_unscaled_1h,
    "rnn": {'MAPE': np.float32(93.520571), 'MAE': np.float32(2.1094285e-08)},
    "lstm": metrics_lstm_unscaled_1h,
    "bi-lstm": metrics_bi_lstm_unscaled_1h,
    "cnn-lstm": metrics_cnn_lstm_unscaled_1h,
    "attention-lstm": metrics_attention_lstm_unscaled_1h,
    "prophet": metrics_prophet_unscaled_1h,
    "arima": metrics_arima_unscaled_1h
}

In [73]:
df_quality_unscaled_1h = pd.DataFrame(results_unscaled_1h).T
df_quality_unscaled_1h.index.name = "Model"

In [74]:
df_quality_unscaled_1h["MAE"] = df_quality_unscaled_1h["MAE"].apply(lambda x: f"{x:.8e}")

In [75]:
styled_df_unscaled_1h = df_quality_unscaled_1h.style.highlight_min(color="lightgreen", axis=0)
styled_df_unscaled_1h

,MAPE,MAE
Model,,
rnn,93.520569,2.10942854e-08
lstm,91.156605,2.08737362e-08
bi-lstm,82.897248,1.99771214e-08
cnn-lstm,84.606403,2.01676065e-08
attention-lstm,83.265488,2.00582019e-08
prophet,149.587151,2.91223640e-08
arima,295.235964,4.45510669e-08


Let's check metrics on the unscaled data the next 5 minutes

In [ ]:
results_unscaled_5m = {
    # "rnn": metrics_rnn_unscaled_5m,
    "rnn": {'MAPE': np.float32(95.23118), 'MAE': np.float32(2.5627902e-08)},
    "lstm": metrics_lstm_unscaled_5m,
    "bi-lstm": metrics_bi_lstm_unscaled_5m,
    "cnn-lstm": metrics_cnn_lstm_unscaled_5m,
    "attention-lstm": metrics_attention_lstm_unscaled_5m,
    "prophet": metrics_prophet_unscaled_5m,
    "arima": metrics_arima_unscaled_5m
}

In [ ]:
df_quality_unscaled_5m = pd.DataFrame(results_unscaled_5m).T
df_quality_unscaled_5m.index.name = "Model"

In [ ]:
df_quality_unscaled_5m["MAE"] = df_quality_unscaled_5m["MAE"].apply(lambda x: f"{x:.8e}")

In [ ]:
styled_df_unscaled_5m = df_quality_unscaled_5m.style.highlight_min(color="lightgreen", axis=0)
styled_df_unscaled_5m

,MAPE,MAE
Model,,
rnn,95.231178,2.56279016e-08
lstm,93.127628,2.13511919e-08
bi-lstm,83.786646,2.03878021e-08
cnn-lstm,85.806217,2.07206821e-08
attention-lstm,83.617822,2.04465925e-08
prophet,150.523393,2.93832579e-08
arima,296.802813,4.48271760e-08
